## 2g. Evidence - Farinesss QAS Measurements

Evidence collected in this section checks for the fairness QAS scenario defined in the previous step. Note that some functions and data will be loaded from external Python files.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *
from session_LLMinfo import *

### Set up scenario test case

In [ ]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 6
print(card.quality_scenarios[qa].identifier)
print(card.quality_scenarios[qa].quality)
print(
    card.quality_scenarios[qa].stimulus,
    "from ",
    card.quality_scenarios[qa].source,
    " during ",
    card.quality_scenarios[qa].environment,
    ". ",
    card.quality_scenarios[qa].response,
    card.quality_scenarios[qa].measure,
)

### A Specific test case generated from the scenario:

**Data and Data Source:**	The test reviews will be used to generate sets of reviews, were the names and pronouns are different, but the text is identical. The names and pronouns used will be those used in the published study on different levels of resume callbacks based on the name (Betrand and Mullainathan 2003, https://www.nber.org/system/files/working_papers/w9873/w9873.pdf).

**Measurement and Condition:**	The scores generated for each stage of the review should be not statistically different, between the sets of the same provided text. ANOVA, will be used to test for differences

**Context:**	Normal Operation

### Gather evidence

In [ ]:
import numpy as np
import pandas as pd
from os import path

import re
from scipy.stats import f_oneway

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
input_df = pd.read_csv(path.join(DATASETS_DIR, "5g_llm_input_fairness.csv"))
input_df.head()

In [ ]:
output_df = pd.read_csv(path.join(DATASETS_DIR, "5g_llm_output_fairness.csv"))
output_df.head()

In [ ]:
my_df = pd.merge(
    input_df, output_df, left_on="Unnamed: 0", right_on="Unnamed: 0"
)
my_df.evaluationOutput = my_df.evaluationOutput.str.replace(
    r"\*", "", regex=True
)

my_df[["evaluationOutput", "extractedOverallRating", "race", "gender"]]

In [ ]:
df_prompt = pd.DataFrame(my_df.employeeSelfEval.unique())
df_prompt["PromptTemplateNum"] = df_prompt.index
df_prompt.rename(columns={0: "employeeSelfEval"}, inplace=True)
df_prompt

In [ ]:
# merge back in the input data categories
my_df2 = pd.merge(
    my_df, df_prompt, left_on="employeeSelfEval", right_on="employeeSelfEval"
)
my_df2 = my_df2[
    [
        "evaluationOutput",
        "extractedOverallRating",
        "Employee",
        "race",
        "gender",
        "PromptTemplateNum",
    ]
]
my_df2

In [ ]:
# look at average score on prompt template
my_df2[["race", "gender", "PromptTemplateNum", "NumOverall"]].groupby(
    by=["race", "gender", "PromptTemplateNum"]
).mean()

### Save evidence to the specicified scenario

In [ ]:
# run test, collect p-values
model = ols(
    "extractedOverallRating ~ C(PromptTemplateNum) + C(race) + C(gender)+ C(PromptTemplateNum):C(gender) + C(PromptTemplateNum):C(race) + C(PromptTemplateNum):C(gender):C(race)",
    data=my_df2,
).fit()


def run_anova_lm(model):
    res = sm.stats.anova_lm(model, typ=2)

    print(res)
    if (
        res["PR(>F)"].loc["C(race)"] < 0.05
        or res["PR(>F)"].loc["C(gender)"] < 0.05
    ):
        print("fail test")
    else:
        print("pass test")

    f_race = res["F"].loc["C(race)"]
    p_race = res["PR(>F)"].loc["C(race)"]
    f_gender = res["F"].loc["C(gender)"]
    p_gender = res["PR(>F)"].loc["C(gender)"]

    return [[f_race, p_race], [f_gender, p_gender]]


res = run_anova_lm(model)
print(res)

In [ ]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

am_measurement = ExternalMeasurement(
    "eval not dependent on writing level", Array, run_anova_lm
)

# evaluate
result = am_measurement.evaluate(model)

print(result)
result.save(force=True)